In [96]:
import sys
from pathlib import Path

repo_dir = Path('..').resolve()
assert repo_dir.name == 'hicetnunc-dataset', repo_dir
if str(repo_dir) not in sys.path:
    sys.path.append(str(repo_dir))

import src.reload; src.reload.reload()

In [77]:
tokens_db = src.utils.read_json(src.config.tokens_db_json_file)
swaps_db = src.utils.read_json(src.config.swaps_db_json_file)

nft_state_log = src.utils.read_json(src.config.nft_state_log_file)
nft_state = src.contracts.nft_state.NFTState()
nft_state_replayer = src.contracts.state_utils.StateReplayer(nft_state_log, nft_state)

ah_state_log = src.utils.read_json(src.config.ah_state_log_file)
ah_state = src.contracts.art_house_state.ArtHouseState()
ah_state_replayer = src.contracts.state_utils.StateReplayer(ah_state_log, ah_state)

money_state_log = src.utils.read_json(src.config.money_state_log_file)
money_state = src.contracts.money_state.MoneyState()
money_state_replayer = src.contracts.state_utils.StateReplayer(money_state_log, money_state)

tr_info_db = src.tr.info_db.TrInfoDB()

nft_state_replayer.replay_to_end()
ah_state_replayer.replay_to_end()
money_state_replayer.replay_to_end()

In [260]:
from collections import Counter
from tqdm import tqdm

tr_hash_to_nft_log_entry = {}

for nft_log_entry in tqdm(nft_state_log):
    entry_row_id = nft_log_entry['row_id']
    entry_tr = tr_info_db.get_full_transaction_by_row_id(entry_row_id)
    assert entry_tr[0]['hash'] not in tr_hash_to_nft_log_entry
    nft_log_entry['hash'] = entry_tr[0]['hash']
    nft_log_entry['volume'] = src.tr.utils.get_ops_volume(entry_tr)
    tr_hash_to_nft_log_entry[entry_tr[0]['hash']] = nft_log_entry

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 48422/48422 [00:11<00:00, 4218.38it/s]


In [261]:
tr_hash_to_ah_log_entry = {}

for ah_log_entry in tqdm(ah_state_log):
    entry_row_id = ah_log_entry['row_id']
    entry_tr = tr_info_db.get_full_transaction_by_row_id(entry_row_id)
    assert entry_tr[0]['hash'] not in tr_hash_to_ah_log_entry
    ah_log_entry['hash'] = entry_tr[0]['hash']
    tr_hash_to_ah_log_entry[entry_tr[0]['hash']] = ah_log_entry

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 46653/46653 [00:11<00:00, 4029.20it/s]


In [262]:
tr_hash_to_money_log_entry = {}

for money_log_entry in tqdm(money_state_log):
    entry_row_id = money_log_entry['row_id']
    entry_tr = tr_info_db.get_full_transaction_by_row_id(entry_row_id)
    assert entry_tr[0]['hash'] not in tr_hash_to_money_log_entry
    money_log_entry['hash'] = entry_tr[0]['hash']
    tr_hash_to_money_log_entry[entry_tr[0]['hash']] = money_log_entry

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15204/15204 [00:05<00:00, 2711.09it/s]


In [263]:
swaps_by_row_id = {}

for swap_entry in swaps_db.values():
    swaps_by_row_id[swap_entry['created_row_id']] = swap_entry

tokens_by_row_id = {}
for token_entry in tokens_db.values():
    tokens_by_row_id[token_entry['mint_ah_row_id']] = token_entry

In [264]:
assert set(tr_hash_to_ah_log_entry.keys()).issubset(set(tr_hash_to_nft_log_entry.keys()))

art_house_addr = 'KT1Hkg5qeNhfwpKW4fXvq7HGZB9z2EnmCCA9'
trash_addr = 'tz1burnburnburnburnburnburnburjAYjjX'

In [302]:
from collections import Counter, defaultdict

author_stats = {}

total_stats = Counter()
total_stats['cur_sold_count'] = 0
total_stats['cur_sold_volume'] = 0
total_stats['cur_author2swap_count'] = 0
total_stats['cur_swap2author_count'] = 0
total_stats['cur_author2other_count'] = 0
total_stats['cur_other2swap_count'] = 0
total_stats['cur_swap2other_count'] = 0
total_stats['cur_other2author_count'] = 0
total_stats['cur_author2trash_count'] = 0
total_stats['cur_other2trash_count'] = 0
total_stats['cur_other2other_count'] = 0


purchases_log = []


for token_entry in tokens_db.values():
    token_entry['cur_sold_count'] = 0
    token_entry['cur_sold_volume'] = 0
    token_entry['cur_author2swap_count'] = 0
    token_entry['cur_swap2author_count'] = 0
    token_entry['cur_author2other_count'] = 0
    token_entry['cur_other2swap_count'] = 0
    token_entry['cur_swap2other_count'] = 0
    token_entry['cur_other2author_count'] = 0
    token_entry['cur_author2trash_count'] = 0
    token_entry['cur_other2trash_count'] = 0
    token_entry['cur_other2other_count'] = 0

    token_entry['cur_author_count'] = 0
    token_entry['cur_swap_count'] = 0
    token_entry['cur_trash_count'] = 0
    token_entry['cur_other_count'] = 0

for swap_entry in swaps_db.values():
    swap_entry['cur_state'] = 'not_created'
    swap_entry['cur_sold_count'] = 0
    swap_entry['closed_stamp'] = -1
    swap_entry['closed_iso_date'] = ''
    swap_entry['closed_row_id'] = -1
    swap_entry['closed_hash'] = ''


for ah_log_entry in ah_state_log:
    entry_method = ah_log_entry['method']
    assert entry_method in ['apply_mint', 'apply_collect', 'apply_swap', 'apply_cancel_swap'], entry_method
    entry_row_id = ah_log_entry['row_id']

    nft_log_entry = tr_hash_to_nft_log_entry[ah_log_entry['hash']]

    if entry_method == 'apply_collect':
        assert nft_log_entry['method'] == 'apply_transfer'
        entry_count = ah_log_entry['count']
        swap_id = ah_log_entry['swap_id']
        swap_entry = swaps_db[str(swap_id)]

    elif entry_method == 'apply_swap':
        assert nft_log_entry['method'] == 'apply_transfer'
        swap_entry = swaps_by_row_id[entry_row_id]

    elif entry_method == 'apply_cancel_swap':
        assert nft_log_entry['method'] == 'apply_transfer'
        swap_id = ah_log_entry['swap_id']
        swap_entry = swaps_db[str(swap_id)]

    elif entry_method == 'apply_mint':
        assert nft_log_entry['method'] == 'apply_mint'
        token_entry = tokens_by_row_id[entry_row_id]

    else:
        assert False


for nft_log_entry in nft_state_log:
    entry_method = nft_log_entry['method']
    assert entry_method in ['apply_transfer', 'apply_mint'], entry_method

    entry_volume = nft_log_entry['volume']
    ah_log_entry = tr_hash_to_ah_log_entry.get(nft_log_entry['hash'])
    money_log_entry = tr_hash_to_money_log_entry.get(nft_log_entry['hash'])

    if entry_method == 'apply_mint':
        assert entry_volume == 0
        assert ah_log_entry['method'] == 'apply_mint'

        entry_token_id = nft_log_entry['token_id']
        entry_count = nft_log_entry['count']
        assert ah_log_entry['count'] == nft_log_entry['count']
        assert ah_log_entry['creator'] == nft_log_entry['creator']
        token_entry = tokens_db[str(entry_token_id)]

        total_stats['cur_author_count'] += entry_count
        token_entry['cur_author_count'] += entry_count
        continue

    for entry_tx in nft_log_entry['txs']:
        tx_from = entry_tx['from']
        tx_to = entry_tx['to']
        tx_token_id = int(entry_tx['token_id'])
        tx_count = int(entry_tx['count'])
        
        if tx_count == 0:
            continue

        token_entry = tokens_db[str(tx_token_id)]
        token_creator = token_entry['creator']
        
        tr_class = None
        
        assert tx_from != tx_to

        if tx_from == token_creator:
            assert entry_volume == 0

            if tx_to == art_house_addr:
                if not ah_log_entry:
                    tr_class = ('author', 'trash')
                    assert entry_volume == 0

                else:
                    tr_class = ('author', 'swap')
                    assert ah_log_entry['method'] == 'apply_swap'
                    assert entry_volume == 0
                    swap_entry = swaps_by_row_id[ah_log_entry['row_id']]
                    assert swap_entry['cur_state'] == 'not_created'
                    swap_entry['cur_state'] = 'active'

            elif tx_to == trash_addr:
                tr_class = ('author', 'trash')
                assert ah_log_entry is None
                assert entry_volume == 0

            else:
                tr_class = ('author', 'other')
                assert ah_log_entry is None
                assert entry_volume == 0

        elif tx_from == art_house_addr:
            if tx_to == token_creator:
                tr_class = ('swap', 'author')
                assert ah_log_entry['method'] == 'apply_cancel_swap'
                assert entry_volume == 0
                swap_entry = swaps_db[str(ah_log_entry['swap_id'])]
                assert swap_entry['cur_state'] == 'active'
                swap_entry['cur_state'] = 'closed'

            elif tx_to == trash_addr:
                assert False

            else:
                tr_class = ('swap', 'other')
                assert ah_log_entry['method'] == 'apply_collect'
                assert entry_volume >= 0
                swap_entry = swaps_db[str(ah_log_entry['swap_id'])]
                assert swap_entry['cur_state'] == 'active'
                assert ah_log_entry['count'] == tx_count

                assert len(nft_log_entry['txs']) == 1

                if swap_entry['price'] == 0:
                    assert money_log_entry is None

                else:
                    assert money_log_entry
                    assert money_log_entry['token_id'] == swap_entry['token_id']
                    assert money_log_entry['token_count'] == tx_count
                    assert abs(round(money_log_entry['price'] * 1e6) - swap_entry['price'] * tx_count) <= 1, (
                        round(money_log_entry['price'] * 1e6),
                        swap_entry['price'],
                    )
                    assert money_log_entry['beneficiary'] == token_creator

                purchases_log.append({
                    'token_id': swap_entry['token_id'],
                    'swap_id': swap_entry['swap_id'],
                    'price': swap_entry['price'],
                    'token_count': tx_count,
                    'token_creator': token_creator,
                })

        else:
            assert entry_volume == 0

            if tx_to == token_creator:
                tr_class = ('other', 'author')
                assert ah_log_entry is None

            elif tx_to == art_house_addr:
                # print(ah_log_entry)
                # print(nft_log_entry)
                # print(token_entry)
                # print()
                tr_class = ('other', 'swap')
                assert ah_log_entry['method'] == 'apply_swap'

            else:
                tr_class = ('other', 'other')
                assert ah_log_entry is None

        sender_class, receiver_class = tr_class
        for stats_entry in [token_entry, total_stats]:
            token_entry[f'cur_{sender_class}_count'] -= tx_count
            token_entry[f'cur_{receiver_class}_count'] += tx_count
            token_entry[f'cur_{sender_class}2{receiver_class}_count'] += tx_count
